# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import pickle
import nltk
from nltk.corpus import stopwords
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('cleaned_messages', engine)
X = df['message']
Y = df.drop(['id','message','original','genre'], axis=1)

In [3]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    # convert text to lowercase
    text = text.lower()
    tokens = word_tokenize(text)
    
    STOPWORDS = list(set(stopwords.words('english')))
    # remove short words
    tokens = [token for token in tokens if len(token) > 2]
    # remove stopwords
    tokens = [token for token in tokens if token not in STOPWORDS]
  
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
def model_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
         
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
model = model_pipeline()
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = model.predict(X_test)

In [8]:
target_names = y_test.columns
target_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [9]:
for i, col in enumerate(y_test.columns.values):
    print("Report for: ", col)
    print(classification_report(y_test.loc[:, col], y_pred[:, i], target_names=target_names))

Report for:  related
                        precision    recall  f1-score   support

               related       0.62      0.45      0.52      2054
               request       0.84      0.91      0.87      6534
                 offer       0.71      0.16      0.26        64

           avg / total       0.78      0.80      0.78      8652

Report for:  request
                        precision    recall  f1-score   support

               related       0.90      0.97      0.93      7180
               request       0.78      0.45      0.57      1472

           avg / total       0.88      0.88      0.87      8652

Report for:  offer
                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      8614
               request       0.00      0.00      0.00        38

           avg / total       0.99      1.00      0.99      8652

Report for:  aid_related
                        precision    recall  f1-score   support

        

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))


In [10]:
for i, col in enumerate(y_test.columns.values):
    print("Scores for: ", target_names[i])
    print("Accuracy: {:.4f}\t\t% Precision: {:.4f}\t\t% Recall: {:.4f}\t\t% F1_score: {:.4f}".format(
            accuracy_score(y_test.loc[:, col], y_pred[:, i]),
            precision_score(y_test.loc[:, col], y_pred[:, i], average='weighted'),
            recall_score(y_test.loc[:, col], y_pred[:, i], average='weighted'),
        f1_score(y_test.loc[:, col], y_pred[:, i], average='weighted')
    ))

Scores for:  related
Accuracy: 0.7979		% Precision: 0.7834		% Recall: 0.7979		% F1_score: 0.7849
Scores for:  request
Accuracy: 0.8848		% Precision: 0.8765		% Recall: 0.8848		% F1_score: 0.8715
Scores for:  offer
Accuracy: 0.9956		% Precision: 0.9912		% Recall: 0.9956		% F1_score: 0.9934
Scores for:  aid_related
Accuracy: 0.7420		% Precision: 0.7410		% Recall: 0.7420		% F1_score: 0.7357
Scores for:  medical_help
Accuracy: 0.9241		% Precision: 0.9059		% Recall: 0.9241		% F1_score: 0.9013
Scores for:  medical_products
Accuracy: 0.9503		% Precision: 0.9361		% Recall: 0.9503		% F1_score: 0.9319
Scores for:  search_and_rescue
Accuracy: 0.9740		% Precision: 0.9632		% Recall: 0.9740		% F1_score: 0.9630
Scores for:  security
Accuracy: 0.9816		% Precision: 0.9820		% Recall: 0.9816		% F1_score: 0.9726
Scores for:  military
Accuracy: 0.9693		% Precision: 0.9573		% Recall: 0.9693		% F1_score: 0.9582
Scores for:  child_alone
Accuracy: 1.0000		% Precision: 1.0000		% Recall: 1.0000		% F1_score: 1.000

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Accuracy: 0.9320		% Precision: 0.9239		% Recall: 0.9320		% F1_score: 0.9234
Scores for:  fire
Accuracy: 0.9896		% Precision: 0.9847		% Recall: 0.9896		% F1_score: 0.9849
Scores for:  earthquake
Accuracy: 0.9649		% Precision: 0.9633		% Recall: 0.9649		% F1_score: 0.9631
Scores for:  cold
Accuracy: 0.9788		% Precision: 0.9718		% Recall: 0.9788		% F1_score: 0.9700
Scores for:  other_weather
Accuracy: 0.9483		% Precision: 0.9297		% Recall: 0.9483		% F1_score: 0.9282
Scores for:  direct_report
Accuracy: 0.8469		% Precision: 0.8331		% Recall: 0.8469		% F1_score: 0.8251


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
model.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [12]:
def model_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
         
    parameters = {        
        'vect__max_df': (0.5, 0.75),
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 100],
    }

    cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1)
    
    return cv

In [13]:
import time
print("Start time: ", time.time())
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
model = model_pipeline()
model.fit(X_train, y_train)
print("End time: ", time.time())

Start time:  1552238880.9081864
End time:  1552246223.5598438


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
y_pred = model.predict(X_test)
target_names = y_test.columns
target_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [15]:
for i, col in enumerate(y_test.columns.values):
    print("Scores for: ", target_names[i])
    print("Accuracy: {:.4f}\t\t% Precision: {:.4f}\t\t% Recall: {:.4f}\t\t% F1_score: {:.4f}".format(
            accuracy_score(y_test.loc[:, col], y_pred[:, i]),
            precision_score(y_test.loc[:, col], y_pred[:, i], average='weighted'),
            recall_score(y_test.loc[:, col], y_pred[:, i], average='weighted'),
        f1_score(y_test.loc[:, col], y_pred[:, i], average='weighted')
    ))

Scores for:  related
Accuracy: 0.8091		% Precision: 0.7948		% Recall: 0.8091		% F1_score: 0.7897
Scores for:  request
Accuracy: 0.8959		% Precision: 0.8907		% Recall: 0.8959		% F1_score: 0.8844
Scores for:  offer
Accuracy: 0.9956		% Precision: 0.9912		% Recall: 0.9956		% F1_score: 0.9934
Scores for:  aid_related
Accuracy: 0.7710		% Precision: 0.7695		% Recall: 0.7710		% F1_score: 0.7696
Scores for:  medical_help
Accuracy: 0.9212		% Precision: 0.8997		% Recall: 0.9212		% F1_score: 0.8909
Scores for:  medical_products
Accuracy: 0.9524		% Precision: 0.9452		% Recall: 0.9524		% F1_score: 0.9346
Scores for:  search_and_rescue
Accuracy: 0.9743		% Precision: 0.9675		% Recall: 0.9743		% F1_score: 0.9628
Scores for:  security
Accuracy: 0.9813		% Precision: 0.9634		% Recall: 0.9813		% F1_score: 0.9722
Scores for:  military
Accuracy: 0.9696		% Precision: 0.9585		% Recall: 0.9696		% F1_score: 0.9582
Scores for:  child_alone
Accuracy: 1.0000		% Precision: 1.0000		% Recall: 1.0000		% F1_score: 1.000

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Accuracy: 0.9375		% Precision: 0.9314		% Recall: 0.9375		% F1_score: 0.9321
Scores for:  fire
Accuracy: 0.9896		% Precision: 0.9846		% Recall: 0.9896		% F1_score: 0.9846
Scores for:  earthquake
Accuracy: 0.9698		% Precision: 0.9687		% Recall: 0.9698		% F1_score: 0.9689
Scores for:  cold
Accuracy: 0.9798		% Precision: 0.9769		% Recall: 0.9798		% F1_score: 0.9715
Scores for:  other_weather
Accuracy: 0.9493		% Precision: 0.9410		% Recall: 0.9493		% F1_score: 0.9270
Scores for:  direct_report
Accuracy: 0.8518		% Precision: 0.8425		% Recall: 0.8518		% F1_score: 0.8281


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            if len(pos_tags) == 0:
                return False
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [17]:
def model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])
         
    parameters = {        
        'features__text_pipeline__vect__max_df': (0.5, 0.75),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 100],
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv

In [18]:
print("Start time: ", time.time())
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
model = model_pipeline()
model.fit(X_train, y_train)
print("End time: ", time.time())

Start time:  1552246622.1439745
End time:  1552250661.36079


In [19]:
y_pred = model.predict(X_test)
target_names = y_test.columns
target_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [20]:
for i, col in enumerate(y_test.columns.values):
    print("Scores for: ", target_names[i])
    print("Accuracy: {:.4f}\t\t% Precision: {:.4f}\t\t% Recall: {:.4f}\t\t% F1_score: {:.4f}".format(
            accuracy_score(y_test.loc[:, col], y_pred[:, i]),
            precision_score(y_test.loc[:, col], y_pred[:, i], average='weighted'),
            recall_score(y_test.loc[:, col], y_pred[:, i], average='weighted'),
        f1_score(y_test.loc[:, col], y_pred[:, i], average='weighted')
    ))

Scores for:  related
Accuracy: 0.7665		% Precision: 0.7439		% Recall: 0.7665		% F1_score: 0.6941
Scores for:  request
Accuracy: 0.8895		% Precision: 0.8814		% Recall: 0.8895		% F1_score: 0.8814
Scores for:  offer
Accuracy: 0.9940		% Precision: 0.9916		% Recall: 0.9940		% F1_score: 0.9928
Scores for:  aid_related
Accuracy: 0.7654		% Precision: 0.7644		% Recall: 0.7654		% F1_score: 0.7611
Scores for:  medical_help
Accuracy: 0.9220		% Precision: 0.9055		% Recall: 0.9220		% F1_score: 0.9095
Scores for:  medical_products
Accuracy: 0.9558		% Precision: 0.9473		% Recall: 0.9558		% F1_score: 0.9480
Scores for:  search_and_rescue
Accuracy: 0.9741		% Precision: 0.9664		% Recall: 0.9741		% F1_score: 0.9683
Scores for:  security
Accuracy: 0.9797		% Precision: 0.9674		% Recall: 0.9797		% F1_score: 0.9724
Scores for:  military
Accuracy: 0.9708		% Precision: 0.9650		% Recall: 0.9708		% F1_score: 0.9668
Scores for:  child_alone
Accuracy: 1.0000		% Precision: 1.0000		% Recall: 1.0000		% F1_score: 1.000

### 9. Export your model as a pickle file

In [21]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.